# Creating Coastal Wetlands Forests Habitat Suitability Dataset 


# Imports

In [2]:
from dataset_builder_original import EEDatasetBuilderOriginal
import geemap
import ee
import os
import numpy as np
import rasterio
import dask.array as da

In [3]:
# Trigger the authentication flow.
ee.Authenticate()


Successfully saved authorization token.


# Map

In [4]:
Map = geemap.Map(center=[29.7666636 , 78.1999992], zoom=7)
Map

Map(center=[29.7666636, 78.1999992], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

# Dataset builder

In [5]:
# Build ee dataset builder
ee_dataset_builder = EEDatasetBuilderOriginal()

In [7]:
# buffer zone for ganga river
ganga_buff = ee.FeatureCollection('projects/nikhilrajdeep/assets/Buffer_Ganga_river')

In [8]:
subsection_clip = ee.FeatureCollection("projects/ee-warnermichael09/assets/gridded_30m_subsection")

In [9]:
subset_naalas = ee.Image('projects/ee-warnermichael09/assets/naalas_masked_30m').clip(subsection_clip)
Map.addLayer(subset_naalas,{} , 'naalas masked')

In [10]:
# Need to upload the masked naala area
ee_dataset_builder.filtered_response_layer_from_raster(
    response_raster="custom", 
    ee_image=subset_naalas,
    custom_response_raster_name='response'
)

### Predictors

In [11]:
merit_water = ee.Image('MERIT/Hydro/v1_0_1').clip(subsection_clip);
ele = ee.Image("NASA/NASADEM_HGT/001");
twi = ee.Image("projects/nikhilrajdeep/assets/TWI_Ganga");
dd = ee.Image("projects/nikhilrajdeep/assets/drainage_density");
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD");
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2");
lc = ee.ImageCollection("ESA/WorldCover/v200");
soil = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02");
pop = ee.ImageCollection("CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density");

gpop = pop.toBands().select('gpw_v4_population_density_adjusted_to_2015_unwpp_country_totals_rev11_2020_30_sec_unwpp-adjusted_population_density').clip(subsection_clip).rename('popDensity');
gpop_cond = gpop.expression(
    "(b('popDensity') > 0 && b('popDensity') < 865) ? 1" +
    ":(b('popDensity') >= 865 && b('popDensity') < 1600) ? 2" +
    ":(b('popDensity') >= 1600 && b('popDensity') < 2680) ? 3" +
    ":(b('popDensity') >= 2680 && b('popDensity') < 4036) ? 4" +
    ":(b('popDensity') >= 4036) ? 5" +
    ": 0"  # Default value in case none of the conditions above are met
).clip(subsection_clip)
conditionParams = {
    'min': 1,
    'max': 5,
    'palette': ['1a9641', 'a6d96a', 'ffffbf', 'fdae61', 'd7191c']
}
Map.addLayer(gpop_cond, conditionParams, 'Population density')
predictors = gpop_cond.rename('PD')

lulc = lc.first().clip(subsection_clip).rename('LULC')

lulcCond = (lulc.eq(80).multiply(5)
            .where(lulc.eq(90), 1)
            .where(lulc.eq(10), 1)
            .where(lulc.eq(20), 1)
            .where(lulc.eq(30), 1)
            .where(lulc.eq(40), 4)
            .where(lulc.eq(60), 2)
            .where(lulc.eq(50), 3))

Map.addLayer(lulcCond, conditionParams, 'LULC');

predictors = predictors.addBands(lulcCond)


In [12]:
rain = chirps.filterDate('2022-01-01','2023-01-01').sum().clip(subsection_clip)
rainCond = (rain
  .where(rain.lt(868), 1)
  .where(rain.gte(868)and(rain.lt(1019)), 2)
  .where(rain.gte(1019)and(rain.lt(1189)), 3)
  .where(rain.gte(1189)and(rain.lt(1410)), 4)
  .where(rain.gte(1410), 5))
Map.addLayer(rainCond, conditionParams, 'Rainfall Categorized');
predictors = predictors.addBands(rainCond)


In [13]:
ddCond = (dd
  .where(dd.lt(0.6), 1)
  .where(dd.gte(0.6)and(dd.lt(1.5)), 2)
  .where(dd.gte(1.5)and(dd.lt(2.6)), 3)
  .where(dd.gte(2.6)and(dd.lt(4)), 4)
  .where(dd.gte(4), 5)).rename('drainage_density').clip(subsection_clip)
Map.addLayer(ddCond, conditionParams, 'drainage density');
predictors = predictors.addBands(ddCond)

In [14]:
dem = ele.select('elevation')
slope = ee.Terrain.slope(dem).clip(subsection_clip)
slopeCond = (slope
  .where(slope.lt(4), 5)
  .where(slope.gte(4)and(slope.lt(12)), 4)
  .where(slope.gte(12)and(slope.lt(24)), 3)
  .where(slope.gte(24)and(slope.lt(35)), 2)
  .where(slope.gte(35), 1))
Map.addLayer(slopeCond, conditionParams, 'slope');
predictors = predictors.addBands(slopeCond)


In [15]:
def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    return image.addBands(opticalBands, None, True)\
                .addBands(thermalBands, None, True)\
                .updateMask(qaMask)\
                .updateMask(saturationMask)
def addindices(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)
ndvi = (l8.filterDate('2022-01-01', '2023-01-01')
        .filterBounds(ganga_buff)
        .filterMetadata('CLOUD_COVER', 'less_than', 10)
        .map(maskL8sr)
        .map(addindices)
        .select('NDVI')
        .mean()
        .clip(ganga_buff))

ndviCond = (ndvi.where(ndvi.lt(0.14), 5)
           .where(ndvi.gte(0.14)and(ndvi.lt(0.33)), 4)
           .where(ndvi.gte(0.33)and(ndvi.lt(0.44)), 3)
           .where(ndvi.gte(0.44)and(ndvi.lt(0.58)), 2)
           .where(ndvi.gte(0.58), 1))



In [16]:
temp = (l8.filterDate('2021-01-01', '2022-01-01')
        .filterBounds(subsection_clip)
        .filterMetadata('CLOUD_COVER', 'less_than', 10)
        .map(maskL8sr)
        .select('ST_B10')
        .map(lambda image: image.subtract(273.15))
        .mean()
        .clip(subsection_clip)).rename('LST')
tempCond = (temp
  .where(temp.lt(25), 1)
  .where(temp.gte(25)and(temp.lt(30)), 2)
  .where(temp.gte(30)and(temp.lt(34)), 3)
  .where(temp.gte(34)and(temp.lt(37)), 4)
  .where(temp.gte(37), 5))
Map.addLayer(tempCond, conditionParams, 'LST 2022');
predictors = predictors.addBands(tempCond)

In [17]:
predictors.bandNames().getInfo()

['PD', 'LULC', 'precipitation', 'drainage_density', 'slope', 'LST']

In [18]:
name_custom_ee_images_list = predictors.bandNames().getInfo()
ee_images_list = [predictors.select(band_name) for band_name in name_custom_ee_images_list]
predictors_list = ['custom_ee_image' for x in name_custom_ee_images_list]

In [19]:
ee_dataset_builder.spatial_covariates(covariates=predictors_list, 
                                      ee_image=ee_images_list,
                                      name_custom_ee_image=name_custom_ee_images_list ) 

In [20]:
ee_dataset_builder.image.bandNames().getInfo()

['response', 'PD', 'LULC', 'precipitation', 'drainage_density', 'slope', 'LST']

Add predictors bands

# Export samples CSV to GCP

In [23]:
# Gridded shapefile asset in GEE
# CCAP west coast 
shp_asset_path = 'projects/ee-warnermichael09/assets/gridded_30m_subsection'

# CCAP south east coast
# shp_asset_path = 'projects/wetlands-lab/assets/ccap_mapping_bndry_gridded_50km_50km_south_east_coast' 

 # CCAP north east coast
# shp_asset_path = 'projects/wetlands-lab/assets/ccap_mapping_bndry_gridded_50km_50km_north_east_coast' 

scale = 30
maxPixels = 1e13
gcp_bucket = 'wetlands-lab'
gcp_folder_name = 'habitat_suitability'
numPoints = 1000 # we override the number of points below with classPoints
classBand = "response"
classPoints = [1, 1]
classValues = [0,1]



In [24]:
samples_folder_name = f'west_coast_ccap_mapping_bndry_gridded_50km_50km_classPoints_4000_1000_classes_13_CCAP_new_climate_data'
# samples_folder_name = f'south_east_coast_ccap_mapping_bndry_gridded_50km_50km_classPoints_4000_1000_classes_13_CCAP_new_climate_data'
# samples_folder_name = f'north_east_coast_ccap_mapping_bndry_gridded_50km_50km_classPoints_4000_1000_classes_13_CCAP_new_climate_data'

# Stratified Sampling
# This will take quite some time
ee_dataset_builder.samples_csv_export(shp_asset_path=shp_asset_path, 
                                      name_gcp_bucket=gcp_bucket, 
                                      folder_in_gcp_bucket=gcp_folder_name + '/' + samples_folder_name, 
                                      scale=scale, 
                                      geometries=True,
                                      isStratifiedSampling=True, 
                                      numPoints=numPoints, 
                                      classValues=classValues,
                                      classBand=classBand, 
                                      classPoints=classPoints)

Geometry number of features: 6007
Stratified sampling: 
numPoints: 1000, 
classBand: response, 
scale: 30, 
geometries: True, 
dropNulls: True, 
tileScale: 1, 
classPoints: [1, 1], 
seed: 0, 
projection:None


  5%|▌         | 318/6007 [58:39<17:26:38, 11.04s/it]

# Export tiles to GCP

In [37]:
################## Export inference tiles to GCP bucket ##################
tiles_folder_name = f'tiles_west_coast_ccap_mapping_bndry_gridded_50km_50km_classes_13_CCAP_scale{scale}_new_climate_data'
# tiles_folder_name = f'tiles_south_east_coast_ccap_mapping_bndry_gridded_50km_50km_classes_13_CCAP_scale{scale}_new_climate_data'
# tiles_folder_name = f'tiles_north_east_coast_ccap_mapping_bndry_gridded_50km_50km_classes_13_CCAP_scale{scale}_new_climate_data'

print(f'\nExport inference tiles using the shapefile: {shp_asset_path}...')
# This will take quite some time
ee_dataset_builder.tiles_export(shp_asset_path,
                                name_gcp_bucket=gcp_bucket,
                                folder_in_gcp_bucket=gcp_folder_name + '/' + tiles_folder_name,
                                maxPixels=maxPixels, scale=scale)
print('Inference tiles export done.\n')


Export inference tiles using the shapefile: projects/wetlands-lab/assets/ccap_mapping_bndry_gridded_50km_50km_west_coast...
Geometry number of features: 402

Starting collecting tiles...


100%|██████████| 402/402 [07:01<00:00,  1.05s/it]

Inference tiles export done.

